In [1]:
import sys
sys.path.append("../")

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from sklearn import feature_extraction
from sklearn import preprocessing

In [3]:
df = pd.read_csv("../data/compas-scores-two-years.csv")

# Filters from mbilalzafar/fair-classification.
df = df.dropna(subset=["days_b_screening_arrest"]) # Dropping missing values.
idx = np.logical_and(df["days_b_screening_arrest"]<=30, df["days_b_screening_arrest"]>=-30)
idx = np.logical_and(idx, df["is_recid"] != -1)
idx = np.logical_and(idx, df["c_charge_degree"] != "O") # F: felony, M: misconduct
idx = np.logical_and(idx, df["score_text"] != "NA")
idx = np.logical_and(idx, np.logical_or(df["race"] == "African-American", df["race"] == "Caucasian"))
df = df[idx]

priors_count = np.reshape(preprocessing.scale(df["priors_count"]), (-1, 1))
age_cat = preprocessing.LabelBinarizer().fit(df["age_cat"]).transform(df["age_cat"])
race = preprocessing.LabelBinarizer().fit(df["race"]).transform(df["race"])
sex = preprocessing.LabelBinarizer().fit(df["sex"]).transform(df["sex"])
c_charge_degree = preprocessing.LabelBinarizer().fit(df["c_charge_degree"]).transform(df["c_charge_degree"])

feature_data = np.hstack((
    priors_count,
    age_cat,
    race,
    sex,
    c_charge_degree
))

target_data = np.reshape(np.array(df["two_year_recid"]), (-1, 1))

train_idxs = range(0, 4278)
test_idxs = range(4278, 5278)

In [4]:
influence_data = np.load("../output/influence_model_on_compas.npz")
influence_values = influence_data["influence_values"]
theta_relatif_values = influence_data["theta_relatif_values"]
l_relatif_values = influence_data["l_relatif_values"]

representer_data = np.load("../output/representer_model_on_compas.npz")
representer_values = representer_data["representer_values"]

In [5]:
(num_training_points, num_test_points) = influence_values.shape
print("{} training points, {} test points.".format(num_training_points, num_test_points))

4278 training points, 1000 test points.


In [6]:
influence_idxs = np.argsort(influence_values, axis=0)
theta_relatif_idxs = np.argsort(theta_relatif_values, axis=0)
l_relatif_idxs = np.argsort(l_relatif_values, axis=0)

# Positive representer values are excitatory points, opposite of influence.
representer_idxs = np.flip(np.argsort(representer_values, axis=0), axis=0)

## Comparing Number of Unique Points

In [7]:
num_test_points_to_compare = num_test_points
num_most_influential_to_include = 5

### Influence Functions

In [8]:
len(np.unique(influence_idxs[:num_most_influential_to_include, :num_test_points_to_compare]))

633

### $\theta$-RelatIF

In [9]:
len(np.unique(theta_relatif_idxs[:num_most_influential_to_include, :num_test_points_to_compare]))

1239

### $l$-RelatIF

In [10]:
len(np.unique(l_relatif_idxs[:num_most_influential_to_include, :num_test_points_to_compare]))

1350

### Representer Points

In [11]:
len(np.unique(representer_idxs[:num_most_influential_to_include, :num_test_points_to_compare]))

201